# 야식(배달) 데이터 전처리
### 주문 지역 주거 특성 & 빈도

In [1]:
import pandas as pd

In [3]:
# 데이터 불러오기
column_names = ['년', '시군구코드','광역시도명','시군구명','세대구성대분류명','세대구성중분류명','세대구성소분류명','단위','빈도']
df = pd.read_csv('data/raw/KGU_2nd_ORIGIN_KGURESIDEINFO_20211013000000.csv', names=column_names, header=None)
df.head()

,년,시군구코드,광역시도명,시군구명,세대구성대분류명,세대구성중분류명,세대구성소분류명,단위,빈도
0,2020,11110,서울특별시,종로구,1세대 가구,부부,일반가구,가구,8752
1,2020,11110,서울특별시,종로구,1세대 가구,부부,주택_단독주택,가구,3162
2,2020,11110,서울특별시,종로구,1세대 가구,부부,주택_아파트,가구,2628
3,2020,11110,서울특별시,종로구,1세대 가구,부부,주택_연립주택,가구,836
4,2020,11110,서울특별시,종로구,1세대 가구,부부,주택_다세대주택,가구,1406


**지역,년도 필터링**

In [ ]:
# 불필요 지역 제거 (서울/경기/인천 외 제외)
# 2020년 데이터만 남기기
filtered_df = df[(df['광역시도명'].isin(['서울특별시', '경기도', '인천광역시'])) & (df['년'] == 2020)]
filtered_df.head()

,년,시군구코드,광역시도명,시군구명,세대구성대분류명,세대구성중분류명,세대구성소분류명,단위,빈도
0,2020,11110,서울특별시,종로구,1세대 가구,부부,일반가구,가구,8752
1,2020,11110,서울특별시,종로구,1세대 가구,부부,주택_단독주택,가구,3162
2,2020,11110,서울특별시,종로구,1세대 가구,부부,주택_아파트,가구,2628
3,2020,11110,서울특별시,종로구,1세대 가구,부부,주택_연립주택,가구,836
4,2020,11110,서울특별시,종로구,1세대 가구,부부,주택_다세대주택,가구,1406


In [ ]:
# 필요없는 컬럼 삭제
filtered_df.drop(columns=['시군구코드','세대구성중분류명','세대구성소분류명','단위'], inplace=True)

In [ ]:
# '다섯가구 미만' → '5'로 대체하여 수치형으로 변환
filtered_df['빈도'].replace({'다섯가구 미만':'5'},inplace=True)

In [ ]:
filtered_df['빈도'] = filtered_df['빈도'].astype(float)
delivery_df = filtered_df

**시군구 통합**

시군구2 : 경기/인천 '구' 단위 → '시' 단위 통합

In [ ]:
def replace_values_with_multiple_substrings(delivery_df, column, replacements):
  for old_value, new_value in replacements.items():
        delivery_df.loc[delivery_df[column].str.contains(old_value, na=False), column] = new_value
  return delivery_df

replacements = {
    '수원시': '수원시',
    '성남시': '성남시',
    '안양시': '안양시',
    '부천시': '부천시',
    '안산시': '안산시',
    '고양시': '고양시',
    '용인시': '용인시',
}
delivery_df['시군구2'] = delivery_df['시군구명']
delivery_df = replace_values_with_multiple_substrings(delivery_df, '시군구2', replacements)

delivery_df

**데이터 저장**

In [60]:
delivery_df.to_csv('data1/delivery.csv')